In [2]:
import sys
sys.path.append('./../src')

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
from bs4 import BeautifulSoup
import requests
import web_scraping as ws
import re
from importlib import reload
from pymongo import MongoClient

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

In [304]:
# This piece of code saves me so much time! 
reload(ws)

<module 'web_scraping' from './../src/web_scraping.py'>

## Scraping Gaysian Diaries ##
Parsing steps:
1. Get all the blog post urls by navigating through each GD page (there are currently 4 pages). Put them in a list.
2. Parse through each blog post and collect the title, post date, and the paragraphs. The
3. Save the titles, post dates in a list of dicts and the paragraphs in .txt files. 

In [119]:
gd_home_page = 'https://gaysiandiaries.com/'

In [305]:
gd_blog_dicts = ws.gd_get_blog_dicts(gd_home_page)

In [311]:
with open('../pickles/gd_blog_dicts.pickle', 'wb') as to_write:
    pickle.dump(gd_blog_dicts, to_write)

## Scraping G3S ##

In [5]:
g3s_post_directory_page = 'https://gaysianthirdspace.tumblr.com/tags'
g3s_soup = ws.create_soup(g3s_post_directory_page)

In [300]:
g3s_blog_urls = ws.g3s_get_blog_urls(g3s_tag_page_urls)

In [303]:
with open('../pickles/g3s_blog_urls.pickle', 'wb') as to_write:
    pickle.dump(g3s_blog_urls, to_write)

In [312]:
g3s_blog_dicts = ws.g3s_get_blog_dicts(g3s_all_blog_urls)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [240]:
with open('../pickles/g3s_blog_dicts.pickle', 'wb') as to_write:
    pickle.dump(g3s_blog_dicts, to_write)

## Loading Gaysian Diaries Google Analytics Data ##

In [316]:
gd_google_analytics_df = pd.read_csv('../data/gd_google_analytics_20160101-20200814.csv', 
                                     skiprows=list(range(5)))

In [317]:
gd_blog_urls = [blog_dict['url'] for blog_dict in gd_blog_dicts]

In [319]:
# Add the base url to all the relative urls
gd_google_analytics_df['Page'] = 'https://gaysiandiaries.com' + gd_google_analytics_df['Page']

In [321]:
# Filter the df for only the blog urls
gd_google_analytics_df = gd_google_analytics_df[gd_google_analytics_df['Page'] \
                                                .isin(gd_blog_urls)].reset_index(drop=True) 

In [329]:
gd_blog_dicts_with_unique_pageviews = []
for blog_dict in gd_blog_dicts:
    blog_dict['unique_pageviews'] = int(gd_google_analytics_df
                                        [gd_google_analytics_df['Page'] == blog_dict['url']]
                                        ['Unique Pageviews'].values[0].replace(',', ''))
    gd_blog_dicts_with_unique_pageviews.append(blog_dict)

## Loading G3S Google Analytics Data ##

In [351]:
g3s_google_analytics_df = pd.read_csv('../data/g3s_google_analytics_20160101-20200814.csv',
                                      skiprows=list(range(5)))

In [352]:
g3s_google_analytics_df['Page'] = 'https://gaysianthirdspace.tumblr.com' + g3s_google_analytics_df['Page']

In [358]:
missing_google_analytics_url = []
for url in g3s_blog_urls:
    if url not in test['Page'].values:
        missing_google_analytics_url.append(url)

In [360]:
missing_google_analytics_url

['https://gaysianthirdspace.tumblr.com/post/136074826195/187-robert-herrick-to-the-virgins-to-make-much',
 'https://gaysianthirdspace.tumblr.com/post/141803998161/family',
 'https://gaysianthirdspace.tumblr.com/post/143867181560/finding-pride-twice-how-i-came-out-as-gay-and',
 'https://gaysianthirdspace.tumblr.com/post/168555319259/do-we-all-need-an-asian-posse',
 'https://gaysianthirdspace.tumblr.com/post/169326527860/rebecca-liu-its-a-process',
 'https://gaysianthirdspace.tumblr.com/post/167924150683/a-mothers-story-of-acceptance',
 'https://gaysianthirdspace.tumblr.com/post/174194103755/call-me-by-my-name-queerness-and-ownership',
 'https://gaysianthirdspace.tumblr.com/post/137444340517/short-story',
 'https://gaysianthirdspace.tumblr.com/post/166340863040/the-whiteness-of-coming-out',
 'https://gaysianthirdspace.tumblr.com/post/167886982521/33-kathys-mom-is-uncomfortable-with-all-this',
 'https://gaysianthirdspace.tumblr.com/post/135674062670/%E7%88%B1%E7%9A%84%E6%B5%B7%E6%B4%8B',


In [361]:
# Filter the df for only the blog urls
g3s_google_analytics_df = g3s_google_analytics_df[g3s_google_analytics_df['Page']
                                                  .isin(g3s_blog_urls)].reset_index(drop=True)

In [363]:
g3s_blog_dicts_with_unique_pageviews = []
for blog_dict in g3s_blog_dicts:
    if blog_dict['url'] in g3s_google_analytics_df['Page'].values:
        blog_dict['unique_pageviews'] = int(g3s_google_analytics_df
                                            [g3s_google_analytics_df['Page'] == blog_dict['url']]
                                            ['Unique Pageviews'].values[0].replace(',', ''))
    g3s_blog_dicts_with_unique_pageviews.append(blog_dict)

In [366]:
counter = 0
for blog_dict in g3s_blog_dicts_with_unique_pageviews:
    if blog_dict['unique_pageviews'] == None:
        counter += 1
counter

29

## Uploading Scraped Data to MongoDB ##

In [142]:
client = MongoClient()
db = client.blogs
db.list_collection_names()

[]

In [376]:
client.get_default_database

<bound method MongoClient.get_default_database of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

In [369]:
db.gd.insert_many(gd_blog_dicts_with_unique_pageviews)

In [370]:
cursor = db.gd.find({}, {'_id': 0, 'title': 1, 'date': 1, 'num_notes': 1, 'unique_pageviews': 1,
                         'url': 1, 'text': 1})

In [371]:
for result in cursor:
    result
    break

{'title': 'Diary Entry #34:\xa0What does reimagining a society that wouldn’t need the police look like?',
 'date': 'August 1st, 2020',
 'num_notes': 3,
 'unique_pageviews': 51,
 'url': 'https://gaysiandiaries.com/post/625293600413171712/diary-entry-34-what-does-reimagining-a-society',
 'text': 'Eight years ago, I lived in Flushing and had recently learned the ways of Korean drumming. One day, I was recruited as a drummer to go to Washington, DC to march and raise awareness about immigration legislation. We left New York City at around 6 am and got back in the evening.\n\nI have a habit of making stupid mistakes that get me into a bit of trouble all the time. When I’ve been tired, hungry, cold, or thirsty, I’ve been even prone to unintentionally cause myself harm. That evening, I was tired as I was walking home alone after all the activities with the contingent. There were around 5 Black or Brown kids hanging around that I had to pass by on my way home. I took a glance at them but avoid

In [372]:
db.g3s.insert_many(g3s_blog_dicts_with_unique_pageviews)

In [373]:
cursor = db.g3s.find({}, {'_id': 0, 'title': 1, 'date': 1, 'num_notes': 1, 'unique_pageviews': 1,
                         'url': 1, 'text': 1})

In [374]:
for result in cursor:
    result
    break

{'title': 'Follower Friday: dandypandy',
 'date': 'Jan 15th, 2016',
 'num_notes': 10,
 'unique_pageviews': 6,
 'url': 'https://gaysianthirdspace.tumblr.com/post/137377584882/follower-friday-dandypandy',
 'text': 'Follower Fridays is a series of profiles highlighting members of Gaysian Third Space to showcase the diversity of gaysians in the Community. This week’s featured member is @dandypandy.\n\n\n\n\n\nWho are you?\n\n\n\nMy identity is in flux at this time of my life as I try to reconcile future aspirations and current actions. I am a second generation Korean American who sometimes forgets to live a little.\n\n\n\nWhere are you from?\n\n\n\nI grew up in the suburbs near LA and am now going to school in San Diego.\n\n\n\nWhat do you do?\n\n\n\nI study mechanical engineering, co-coordinate a student organization, volunteer in river clean-ups, and work with GIS. In my free time I like to take pictures, eat froyo, and dance (after having a few drinks).\n\n\n\nWhat are you passionate ab

In [184]:
pipeline = [{'$match': {'title': 'Diary Entry #33: How real are dreams?'}}, {'$count': 'test_count'}]

list(db.gd.aggregate(pipeline))

[{'test_count': 1}]